<a href="https://colab.research.google.com/github/klarcika/TZ/blob/main/Naloga8_Navodila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#gručenje
import pandas as pd
df= read_csv("housing_predp.csv")

In [ ]:
stevilski=df.select_dtypes(exclude=["object"]).columns
df[stevilski]

In [1]:
st_gruc=2

In [ ]:
from sklearn.cluster import KMeans
dfSt=df[stevilski]
km= KMeans(n_clusters=st_gruc, random_state=0)
km.fit(dfSt)
km.labels_ #v katero gruco je razdelil prvih in zadnjih nekaj instanc

In [ ]:
import collections
collections.Counter(km.labels_) #preštejje koliko jih je v eni in koliko v dugi gruči

In [ ]:
import seaborn as sns
sns.relplot(x="Price", y="YearBuild", data=dfSt, hue=km.labels_, alpha=.8, height=6)#z hue obarvamo, treutno instance v gručo katero so bile razdeljene
#upoštevamo katera delitev najbolj vpliva na dellitev gruče

In [ ]:
sns.relplot(x="DistanceFromCenter", y="YearBuild", data=dfSt, hue=km.labels_, alpha=.8, height=6)#z hue obarvamo, treutno instance v gručo katero so bile razdeljene

In [ ]:
#kaj pa če bi dodali vse podatke not, pri gručenju lahko pokažemo razmerja med vsemi podatki => uporabimo dekompozicijo nasih podatkov
from sklearn.decomposition import FastICA#vse stolpce bomo prikazali znotraj enega grafa=> celoten dataset spremenimo v 2 stolpca
tr=FastICA(n_components=2).fit_transform(dfSt) #kot izhod 2 stolpca
#! slabost ne obdrži strukture dataframe-a
dfTr=pd.DataFrame(tr, columns=("col1", "col2"))
dfTr.head()

In [ ]:
sns.relplot(x="col1", y="col2", data=dfTr, hue=km.labels_, alpha=.8, height=6) #km.labels_ <- je še netransformirani podatki

In [ ]:
#še 1x naredimo gručenje nad transformiranimi podatki
from sklearn.cluster import KMeans
dfSt=df[stevilski]
km= KMeans(n_clusters=3, random_state=0)
km.fit(dfSt)
km.labels_ #v katero gruco je razdelil prvih in zadnjih nekaj instanc

In [ ]:
#še enkrat izrisemo graf
sns.relplot(x="col1", y="col2", data=dfTr, hue=km.labels_, alpha=.8, height=6) #km.labels_ <- je še netransformirani podatki


In [ ]:
#če se hocemo odlocit na koliko gruc naj delimo
v_in={} #inercija je razdalja med gručo in njenim razdelkom
for i in range(1,10):#testiramo katero deljenje gruc bi bilo najbolj primerno
  km2=KMeans(n_clusters=i, random_state=0, n_init="auto")
  km2.fit(dfTr)
  v_in[i]=km2.inertia_ #shranimo  dictionary, imamo vrednost: ključ

In [ ]:
import matplotlib.pyplot as plt
sns.lineplot(x= list(v_in.keys()), y=list(v_in.values()))
plt.ylabel("inercija")
plt.xlabel("število gruč")
#kako se na podlagi grafa odlocimo katera delitev je najboljsa
#pravilo komolca - gledamo prvi večji prelom in tam je najbolj optimalno št. gruč
#zakaj ne vzamemo 9, ko je blizu 0 => ker bi bila vsaka instanca v svoji gruči
#iščemo st gruc ki nam bo pomensko razdelilo instance

In [ ]:
#lahko pa si pomagamo tudi z dendrogramom
#računamo razdalje, ki jih uporabimo znotraj dendrograma=> razdalje med različnimi gručami
from scipy.cluster.hierarchy import linkage, dendrogram
razdalje=linkage(dfSt, "single")
dendrogram(razdalje, orientation="top", p=5, turncate_mode="level")
          #navpičen, vodoraven; do katerega nivoja ga razgradimo
#dolžina črte predstavlja razdaljo med gručami
#številke na x osi spodaj predstavlja št primerkov;
#številka v oklepaju (x) predstavlja, da je v tej gruči samo ena instanca in tista (x) , x=id